In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [3]:
data_review = pd.read_parquet("dataSet/user_review_clean.parquet")

In [4]:
data_review.head(5)

,item_id,feeling_negative,feeling_positive,feeling_neutral
0,10,2,34,20
1,10090,6,39,6
2,10130,0,2,0
3,10140,1,0,0
4,10150,1,7,1


In [5]:
data_games = pd.read_parquet("dataSet/steam_game_clean.parquet")

In [6]:
data_games.head(5)

,publisher,genres,app_name,title,price,early_access,id,developer,Year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0


In [7]:
#toma una columna 'title', para filtrar todos los duplicados luego tomamos el primer valor que nos da.
data_games.drop_duplicates(subset='title', keep='first', inplace=True)

In [403]:
data_games

,publisher,genres,app_name,title,price,early_access,id,developer,Year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
5,"Making Fun, Inc.",Free to Play,Ironbound,Ironbound,0.00,False,643980,Secret Level SRL,2018.0
9,Poolians.com,Casual,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,False,670290,Poolians.com,2017.0
14,彼岸领域,Action,弹炸人2222,弹炸人2222,0.99,False,767400,彼岸领域,2017.0
17,Trickjump Games Ltd,Action,Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018.0
...,...,...,...,...,...,...,...,...,...
71485,Bidoniera Games,Action,Kebab it Up!,Kebab it Up!,1.99,False,745400,Bidoniera Games,2018.0
71489,Ghost_RUS Games,Casual,Colony On Mars,Colony On Mars,1.99,False,773640,"Nikita ""Ghost_RUS""",2018.0
71493,Sacada,Casual,LOGistICAL: South Africa,LOGistICAL: South Africa,4.99,False,733530,Sacada,2018.0
71496,Laush Studio,Indie,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0


In [8]:
#cambiar el nombre de la columna 'id' por el de 'item_id'
data_games.rename(columns={'id': 'item_id'}, inplace=True)

In [9]:
data_games

,publisher,genres,app_name,title,price,early_access,item_id,developer,Year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018.0
5,"Making Fun, Inc.",Free to Play,Ironbound,Ironbound,0.00,False,643980,Secret Level SRL,2018.0
9,Poolians.com,Casual,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,False,670290,Poolians.com,2017.0
14,彼岸领域,Action,弹炸人2222,弹炸人2222,0.99,False,767400,彼岸领域,2017.0
17,Trickjump Games Ltd,Action,Battle Royale Trainer,Battle Royale Trainer,3.99,False,772540,Trickjump Games Ltd,2018.0
...,...,...,...,...,...,...,...,...,...
71485,Bidoniera Games,Action,Kebab it Up!,Kebab it Up!,1.99,False,745400,Bidoniera Games,2018.0
71489,Ghost_RUS Games,Casual,Colony On Mars,Colony On Mars,1.99,False,773640,"Nikita ""Ghost_RUS""",2018.0
71493,Sacada,Casual,LOGistICAL: South Africa,LOGistICAL: South Africa,4.99,False,733530,Sacada,2018.0
71496,Laush Studio,Indie,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0


In [10]:
data_review = pd.merge(data_review, data_games, on = 'item_id', how= 'inner', indicator=True)

In [11]:
data_review = data_review.sample(n=1000, random_state=40)

In [12]:
data_review.head()

,item_id,feeling_negative,feeling_positive,feeling_neutral,publisher,genres,app_name,title,price,early_access,developer,Year,_merge
2990,72840,0,1,0,Bethesda Softworks,Action,Fallout New Vegas®: Gun Runners’ Arsenal™,Fallout New Vegas®: Gun Runners’ Arsenal™,1.99,False,Obsidian Entertainment,2011.0,both
2400,393530,0,1,1,PlayFlock,Adventure,Insanity Clicker,Insanity Clicker,0.00,False,PlayFlock,2016.0,both
314,210898,1,1,0,Paradox Interactive,Strategy,Crusader Kings II: Mediterranean Portraits,Crusader Kings II: Mediterranean Portraits,1.99,False,Paradox Development Studio,2013.0,both
2575,41700,0,8,1,GSC Game World,Action,S.T.A.L.K.E.R.: Call of Pripyat,S.T.A.L.K.E.R.: Call of Pripyat,19.99,False,GSC Game World,2010.0,both
1830,335000,0,1,0,OhNoo Studio,Adventure,Tormentum - Dark Sorrow,Tormentum - Dark Sorrow,11.99,False,OhNoo Studio,2015.0,both


In [13]:
data_review.shape

(1000, 13)

In [14]:
data_tmp = data_review[['genres', 'feeling_negative', 'feeling_positive', 'feeling_neutral', 'item_id']]

In [15]:
data_tmp.dtypes

genres              object
feeling_negative     int64
feeling_positive     int64
feeling_neutral      int64
item_id             object
dtype: object

In [28]:
data_tmp.head(5)

,genres,feeling_negative,feeling_positive,feeling_neutral,item_id
1942,Indie,0,3,0,344770
496,Action,1,2,0,224960
1004,Indie,18,18,2,262060
2463,Action,0,1,0,401810
2739,Indie,0,1,0,449710


In [17]:
#data_review.drop(['_merge'], inplace=True, axis=1)

In [18]:
data_tmp.isnull().sum()

genres              0
feeling_negative    0
feeling_positive    0
feeling_neutral     0
item_id             0
dtype: int64

In [19]:
data_tmp.duplicated().sum()

0

In [20]:
data_tmp.drop_duplicates(inplace=True) 

C:\Users\BORDADO Y ESTAMPADO\AppData\Local\Temp\ipykernel_3908\4134813729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp.drop_duplicates(inplace=True)


In [21]:
data_tmp = data_tmp.sample(n=1000, random_state=38)

In [22]:
#si el archivo base_modelo_clean.parquet NO EXISTE debe de omitirse este paso. Continuar en el vectorizer
data_review = pd.read_parquet("./dataSet/base_modelo_clean.parquet")

In [23]:
data_tmp.head(5)

,genres,feeling_negative,feeling_positive,feeling_neutral,item_id
1942,Indie,0,3,0,344770
496,Action,1,2,0,224960
1004,Indie,18,18,2,262060
2463,Action,0,1,0,401810
2739,Indie,0,1,0,449710


In [24]:
data_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1942 to 1324
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            1000 non-null   object
 1   feeling_negative  1000 non-null   int64 
 2   feeling_positive  1000 non-null   int64 
 3   feeling_neutral   1000 non-null   int64 
 4   item_id           1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


In [25]:
# Convertir 'item_id' de tipo 'object' a 'int'
data_tmp['item_id'] = data_tmp['item_id'].astype(int)

In [26]:
data_tmp.shape

(1000, 5)

In [27]:
# Primero, vamos a convertir la columna 'nombre' a una representación numérica usando TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_tmp['genres'])

# Luego, vamos a añadir las columnas numéricas a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(), data_tmp['feeling_negative'], data_review['feeling_positive'], data_review['feeling_neutral'],])


KeyError: 'feeling_positive'

In [433]:
features

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0.,  0., ...,  1.,  1.,  1.],
       [ 0.,  0.,  0., ..., 18.,  1.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  2., 65., 16.],
       [ 1.,  0.,  0., ...,  1.,  4.,  1.],
       [ 0.,  0.,  0., ...,  0.,  4.,  0.]])

In [434]:
# Reindexamos el DataFrame
data_review = data_review.reset_index(drop=True)



In [435]:
# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)



In [437]:
# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
id_game = 344770  # El nombre del producto para el que quieres hacer recomendaciones
producto = data_tmp[data_tmp['item_id'] == id_game]

if not producto.empty:
    product_index = producto.index[0]
    product_similarities = similarity_matrix[product_index]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data_review.loc[most_similar_products_indices, 'item_name']
    print("Los productos más similares al producto", id_game, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")


Producto no encontrado


In [1]:
# Para hacer recomendaciones, puedes buscar los productos más similares a un producto dado
id_user = 'diet_cola'  # El nombre del producto para el que quieres hacer recomendaciones
producto = data_review[data_review['user_id'] == id_user]  # Asegúrate de que 'user_id' sea el nombre correcto de la columna

if not producto.empty:
    product_index = producto.index[0]
    product_similarities = similarity_matrix[product_index]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data_review.loc[most_similar_products_indices, 'genres']
    print("Los productos más similares al producto", id_user, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")


NameError: name 'data_review' is not defined

In [352]:
data_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   review             1000 non-null   object 
 1   item_id            1000 non-null   int32  
 2   recommend          1000 non-null   bool   
 3   sentimentanalysis  1000 non-null   float64
 4   item_name          1000 non-null   object 
 5   user_id            1000 non-null   object 
dtypes: bool(1), float64(1), int32(1), object(3)
memory usage: 36.3+ KB


In [350]:
modelo = './dataSet/base_modelo_clean.parquet'
data_review.to_parquet(modelo, index=False)
print(f'Se guardó el archivo {modelo}')

Se guardó el archivo ./dataSet/base_modelo_clean.parquet


In [347]:
most_similar_products[:5]

806              Tomb Raider
5                Tomb Raider
520              Tomb Raider
309    Saints Row: The Third
305        BioShock Infinite
Name: item_name, dtype: object

In [373]:
def recomendacion_juego(id_de_producto:int):
    modelo = pd.read_parquet('./dataSet/base_modelo_clean.parquet')
    #Primero, vamos a convertir la columna 'item_name' a una representación numérica usando TF-IDF (Frecuencia de Terminos - Freciencia Inversa de Terminos)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(modelo['item_name'])

    #se buscan los que son similares al id del juego
    features = np.column_stack([tfidf_matrix.toarray(), modelo['recommend'], modelo['sentimentanalysis']])

    #Verificamos la existencia del Id del juego a establecer simulitud de juegos
    result = modelo[modelo['item_id'] == id_de_producto]
    nombre_del_juego=result.iloc[0]['item_name']
    if result.empty:
        return "No hay datos para el datos para el juego seleccionado."
    #Reindexamos el DataFrame
    data_aplicativo = modelo.reset_index(drop=True)
    
    #Ahora, calculamos la matriz de similitud de coseno
    similarity_matrix = cosine_similarity(features)

    # Para hacer recomendaciones, puedes buscar los juegos más similares a un juego dado
    juego = data_aplicativo[data_aplicativo['item_name'] == nombre_del_juego].index[0]
    score = list(enumerate(similarity_matrix[juego]))
    score= sorted(score, key=lambda x: x[1],reverse=True)
    resultado = score[1:6]
    total = modelo['item_name'].iloc[[i[0] for i in resultado]].tolist()
    
    return{'Juego Recomendado ': total}

    


In [374]:
recomendacion_juego(230410)

{'Juego Recomendado ': ['Warframe',
  'Warframe',
  'Warframe',
  'Warframe',
  'Warframe']}